In [1]:
import requests as r
import shutil
import splinter
from bs4 import BeautifulSoup
import os
import json
import csv

In [2]:
#眼線筆

import requests as r
import shutil
from bs4 import BeautifulSoup
import os
import json
import csv

product_list=[]
i=1
while True:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_index=r.get('https://www.urcosme.com/tags/119/products?page=%s&channel=0&effect=0&texture=0&modus=0&market_date=0&is_limit=false&is_discontinued=false&is_withdraw=false&is_cosme_store=false&price_min=&price_max=&sort=2&is_ajax=1'%i,headers=headers)
    soup=BeautifulSoup(product_index.text,'html5lib')
    if soup.select('div.uc-product-item > div.product-infomation > div.product-name > a') == []:
        break
    for page in soup.select('div.uc-product-item > div.product-infomation > div.product-name > a'):
        product_list.append('https://www.urcosme.com'+page['href'])
#        print(page)
#    print(i)
    i+=1
    
    
for url in product_list:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_page=r.get(url,headers=headers)
    soup=BeautifulSoup(product_page.text,'html5lib')
    comment_list=[]


    product={}
    product['product_name']=soup.select('div.headline-title.product-name')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    
    product['brand']=soup.select('div.brand-name > a.uc-main-link')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    for i in range(0,len(soup.select('div.other-label'))):
        if soup.select('div.other-label')[i].text=='容量':
            product['volumn']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='價格':
            product['price']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='上市日期':
            product['listing_date']=soup.select('div.other-text')[i].text
    if soup.select('div.product-info-image > img.img-auto-center')[0]['src']=='https://dg9ugnb21lig7.cloudfront.net/uploads/images/product_default_250x250.png':
        product['pro_picture']='null'
    else:
        product['pro_picture']=soup.select('div.product-info-image > img.img-auto-center')[0]['src']
    product['label']=[]
    for i in soup.select('div.detail-text'):
        product['label'].append(i.text)
    try:
        product['pro_info']=soup.find('div','product-desc-content').text
    except:
        product['pro_info']='null'
    try:
        product['score']=soup.find('div','score-txt').text.split(' ')[1]
    except:
        product['score']='null'
    try:    
        if soup.select('div.uc-container-sub-title')[0].text=='色號':
            color_code = [i.text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','') for i in soup.select('div.color-tag-container > div.color-number-items > div.color-number-item > div.color-number-item-text > div.two-line-dot')]
        else:
            color_code=['one_color']
    except:
        color_code=['one_color']
    #if os.path.exists('D:\\產品資料\\產品\\唇膏\\'+product['brand']+'_'+product['product_name']+'_'++'.json'):
    #    continue    
    
    i=1
    comment_list=[]
    while True:
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comment_list_url=r.get(url+'/reviews?page=%s'%i,headers=headers)
        comment_soup=BeautifulSoup(comment_list_url.text,'html5lib')
        if comment_soup.select('div.uc-reviews > a') ==[]:
            break
        for a in comment_soup.select('div.uc-reviews > a'):
            comment_list.append('https://www.urcosme.com'+a['href'])
        i+=1
    
    print('comment_list complete')

    product['comments']=[]
    for count in range(0,len(comment_list)):
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comments=r.get(comment_list[count],headers=headers)
        soup=BeautifulSoup(comments.text,'html5lib')
        artist_INFO=soup.select('div.author-name > a')
        artist_ID=artist_INFO[0]['href'].split('/')[2]
        artist_name=artist_INFO[0].text
        review_content=soup.select('div.review-content')[0].text
        affect=soup.find('div','review-effects').text.split('：')[1].split('、')
        user_score=soup.find('div','review-score').text
        comments_published_time=soup.find('div','review-date-and-pageview').text.split(' ')[0]
        product['comments'].append({'user_ID':artist_ID,
                                     'content':review_content,
                                     'affect':affect,
                                     'user_score':user_score,
                                     'comments_published_time':comments_published_time})
    print('評論已完成')
    for col in color_code:
        product['product_color']=col
        with open('D:\\產品資料\\產品\\眼線筆\\'+product['brand']+'_'+product['product_name']+'_'+col+'.json','w') as f :
            json.dump(product,f)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


['https://www.urcosme.com/products/29923',
 'https://www.urcosme.com/products/23258',
 'https://www.urcosme.com/products/7543',
 'https://www.urcosme.com/products/47719',
 'https://www.urcosme.com/products/7416',
 'https://www.urcosme.com/products/42844',
 'https://www.urcosme.com/products/6470',
 'https://www.urcosme.com/products/14852',
 'https://www.urcosme.com/products/9927',
 'https://www.urcosme.com/products/45691',
 'https://www.urcosme.com/products/42372',
 'https://www.urcosme.com/products/9556',
 'https://www.urcosme.com/products/8481',
 'https://www.urcosme.com/products/52239',
 'https://www.urcosme.com/products/11450',
 'https://www.urcosme.com/products/8702',
 'https://www.urcosme.com/products/11219',
 'https://www.urcosme.com/products/37504',
 'https://www.urcosme.com/products/75494',
 'https://www.urcosme.com/products/73445',
 'https://www.urcosme.com/products/14415',
 'https://www.urcosme.com/products/16485',
 'https://www.urcosme.com/products/14775',
 'https://www.urco

In [11]:
#某產品

for url in product_list:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_page=r.get(url,headers=headers)
    soup=BeautifulSoup(product_page.text,'html5lib')
    comment_list=[]


    product={}
    product['product_name']=soup.select('div.headline-title.product-name')[0].text.replace('?',' ').replace('\u3000',' ').replace('♥',' ').replace('@',' ').replace('"',' ').replace('.',' ').replace(':',' ').replace('!',' ').replace('/',' ').replace('*',' ').replace('<',' ').replace('>',' ').replace('(',' ').replace(')',' ').replace('\t',' ').replace('/',' ')   
    
    product['brand']=soup.select('div.brand-name > a.uc-main-link')[0].text
    for i in range(0,len(soup.select('div.other-label'))):
        if soup.select('div.other-label')[i].text=='容量':
            product['volumn']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='價格':
            product['price']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='上市日期':
            product['listing_date']=soup.select('div.other-text')[i].text
    if soup.select('div.product-info-image > img.img-auto-center')[0]['src']=='https://dg9ugnb21lig7.cloudfront.net/uploads/images/product_default_250x250.png':
        product['pro_picture']='null'
    else:
        product['pro_picture']=soup.select('div.product-info-image > img.img-auto-center')[0]['src']
    product['label']=[]
    for i in soup.select('div.detail-text'):
        product['label'].append(i.text)
    try:
        product['pro_info']=soup.find('div','product-desc-content').text
    except:
        product['pro_info']='null'
    try:
        product['score']=soup.find('div','score-txt').text.split(' ')[1]
    except:
        product['score']='null'
    try:    
        if soup.select('div.uc-container-sub-title')[0].text=='色號':
             color_code = [i.text for i in soup.select('div.color-tag-container > div.color-number-items > div.color-number-item > div.color-number-item-text > div.two-line-dot')]
        else:
            color_code=['one_color']
    except:
        color_code=['one_color']
    #if os.path.exists('D:\\產品資料\\產品\\眼線筆\\'+product['brand']+'_'+product['product_name']+'_'++'.json'):
    #    continue    
    
    i=1
    comment_list=[]
    while True:
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comment_list_url=r.get(url+'/reviews?page=%s'%i,headers=headers)
        comment_soup=BeautifulSoup(comment_list_url.text,'html5lib')
        if comment_soup.select('div.uc-reviews > a') ==[]:
            break
        for a in comment_soup.select('div.uc-reviews > a'):
            comment_list.append('https://www.urcosme.com'+a['href'])
        i+=1
    
    print('comment_list complete')

    product['comments']=[]
    for count in range(0,len(comment_list)):
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comments=r.get(comment_list[count],headers=headers)
        soup=BeautifulSoup(comments.text,'html5lib')
        artist_INFO=soup.select('div.author-name > a')
        artist_ID=artist_INFO[0]['href'].split('/')[2]
        artist_name=artist_INFO[0].text
        review_content=soup.select('div.review-content')[0].text
        affect=soup.find('div','review-effects').text.split('：')[1].split('、')
        user_score=soup.find('div','review-score').text
        comments_published_time=soup.find('div','review-date-and-pageview').text.split(' ')[0]
        product['comments'].append({'user_ID':artist_ID,
                                     'content':review_content,
                                     'affect':affect,
                                     'user_score':user_score,
                                     'comments_published_time':comments_published_time})
        print(count)
    for col in color_code:
        product['product_color']=col
        with open('D:\\產品資料\\產品\\眼線筆\\'+product['brand']+'_'+product['product_name']+'_'+col+'.json','w') as f :
            json.dump(product,f)
    
#for i in product['comments']:

comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
comment_list complete
0
1
2
3
4
5
6
7
8
9

18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
comment_list complete
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
comment_list complete
0
1

0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
1
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complete
0
comment_list complet

In [8]:
product

{'brand': 'Solone',
 'comments': [{'affect': ['會回購',
    '用量省',
    '價格實在',
    '好上手',
    '攜帶方便',
    '防水',
    '好搭配',
    '不暈染',
    '不積線',
    '濃密',
    '筆芯軟'],
   'comments_published_time': '2015.08.06',
   'content': '這一款真的讓我不知道該怎麼評啊\n因為有時候覺得很好用，有時候又覺得踩到雷算是產品品質比較不穩定...或許是跟個人習慣有關係目前手上有的顏色試在手上的狀況，可以看出顯色度很夠防水測試的部分也很OK，沖水約30秒後，搓了還是不會掉色卸妝的部分，用卸妝油就可以很輕易的卸掉嚕我自己用眼唇專用卸妝液也很OK金鑽黑是我最愛用的眼線色，不會黑得太過分筆芯又不會太軟，防水又不暈染價格實在，非常適合新手使用的眼膠筆之前買過咖啡色，但是很快就乾掉了...讓我就沒回購了藍色跟紫色還OK，之前買來搭配眼影使用的最近買的濃密黑...嚴重的讓我覺得我買到雷因為筆芯太軟，很容易就會畫得太粗或暈開....讓我一直很懷疑到底是顏色問題還是平均品質的問題....所以總結上面這些經驗我決定給五顆心畢竟它還是一款CP值很高產品就是',
   'user_ID': '399359',
   'user_score': '5'},
  {'affect': ['顯色', '持久', '不暈染', '防水', '不積線'],
   'comments_published_time': '2017.09.01',
   'content': '來自朋友的禮物外觀:筆蓋式 全黑筆芯: 質地軟 (所以也削不尖) ，也因此不能太用力畫(最上面第一條 我只用稍重的力度 試試 能有多深多均勻 \xa0~~~筆芯就裂了中間就輕輕掃了好幾條最後一個就適中力道 粗細大概類似於手上的血管)持久度基本上平常手摸眼睛的力度 \xa0 完全不會掉下一步我就用手來回摩擦到皮膚感覺熱的程度就搓掉9成(所以千萬別揉眼睛)防水度超好的!XD水珠完全避開衛生紙輕壓吸水即可補9/3: 這支是 第一代產品，不是最近新出的第二代升級版(名字實在太像了，好難分阿)',
   'user_ID': '442795',

In [ ]:
#all comment url of  one product

#browser = splinter.Browser('chrome',executable_path='D:\\Downloads\\chromedriver.exe')
#browser.visit('https://www.urcosme.com/products/29923/reviews')
i=1
comment_list=[]
while True:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    comment_list_url=r.get('https://www.urcosme.com/products/29923/reviews?page=%s'%i,headers=headers)
    comment_soup=BeautifulSoup(comment_list_url.text,'html5lib')
    if comment_soup.select('div.uc-reviews > a') ==[]:
        break
    for a in comment_soup.select('div.uc-reviews > a'):
        comment_list.append('https://www.urcosme.com'+a['href'])
    i+=1


In [ ]:
#product裡的comments

product['comments']=[]
for count in range(0,len(comment_list)):
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    comments=r.get(comment_list[count],headers=headers)
    soup=BeautifulSoup(comments.text,'html5lib')
    artist_INFO=soup.select('div.author-name > a')
    artist_ID=artist_INFO[0]['href'].split('/')[2]
    artist_name=artist_INFO[0].text
    review_content=soup.select('div.review-content')[0].text
    affect=soup.find('div','review-effects').text.split('：')[1].split('、')
    user_score=soup.find('div','review-score').text
    comments_published_time=soup.find('div','review-date-and-pageview').text.split(' ')[0]
    product['comments'].append({'user_ID':artist_ID,
                                 'content':review_content,
                                 'affect':affect,
                                 'user_score':user_score,
                                 'comments_published_time':comments_published_time})

In [ ]:
product['comments']=[{'user_ID':artist_ID,
                      'content':review_content,
                      'affect':affect,
                      'user_score':user_score,
                      'comments_published_time':comments_published_time}]

In [10]:
with open('C:\\Users\\willy\\Desktop\\專題\\'+product['product_name']+'.json','w') as f :
    json.dump(product,f)

In [30]:
import json

with open('D:\\Downloads\\MAYBELLINE 媚比琳_好氣色漸層三色CC輕唇膏_S BK01.json','r') as f:
    a=json.load(f)
a

{'brand': 'MAYBELLINE 媚比琳',
 'comments': [{'affect': ['自然',
    '會回購',
    '好上手',
    '好搭配',
    '滋潤',
    '不黏膩',
    '改善唇紋',
    '立體',
    '順滑'],
   'comments_published_time': '2017.07.31',
   'content': '今天要分享的是Maybelline的好氣色漸層三色CC輕唇膏，色號是RD03 野莓冰沙打開來的唇膏切面是這樣，主色調範圍最大，是偏紫深紅色，過渡色是桃紅，最外層是淺粉色首先來個手部試色，光源充足處拍攝，看出它是屬於具有光澤感的唇膏上唇：原本有唇色不均或唇周暗沉的人可以先做唇部遮瑕，再上唇膏漸層會比較明顯，不受原深唇色的影響若本身唇色較淺的人可以直接上唇膏，如下結論：這款唇膏為開價唇膏中的中等價位，它塗上後有潤澤的效果，不會讓嘴唇整天都乾乾的持久度的方面普通，吃完東西漸層通常會受影響，還是要再補唇膏不過如果唇況不好、有死皮的時候，盡量不要塗這支，因為它會讓你的死皮更明顯...顏色自然，也適合上班上課或平常淡妝的時候使用，不過嘴唇較薄的人，可能比較難做出咬唇的漸層效果外層顏色很淺，如果本身唇色比較深或唇周暗沉的人記得做好唇部遮瑕上完的漸層顏色我覺得非常美，而且讓嘴唇更立體了，滋潤、不顯唇紋，也很開心選擇了RD03，是我喜歡的風格～總結來說，這款唇膏我還是會推薦！',
   'user_ID': '382584',
   'user_score': '6'},
  {'affect': ['保濕',
    '溫和低刺激',
    '改善唇紋',
    '顯色',
    '遮蓋唇色',
    '攜帶方便',
    '不引起過敏',
    '滋潤',
    '不黏膩',
    '自然',
    '好推勻',
    '好上手',
    '妝感自然'],
   'comments_published_time': '2017.03.21',
   'content': '哈囉我是yo這次要介紹的是maybelline的漸層唇膏其實他剛出來的時候我就買了只是一直沒有好好用他因為最近才學會擦唇膏哈哈哈覺得他的外殼比第一代CC輕唇膏好看比較有質感 不然太像玩

In [18]:
res=r.get('https://www.urcosme.com/products/58054',headers=headers)
soup=BeautifulSoup(res.text,'html5lib')
soup.select('div.uc-container-sub-title')
if soup.select('div.uc-container-sub-title')[0].text=='色號':
    a=[i.text for i in soup.select('div.color-tag-container > div.color-number-items > div.color-number-item > div.color-number-item-text > div.two-line-dot')]
else:
    print('no')
print(a)

['B101', 'BP402', 'BP501', 'C402', 'CR402', 'M401', 'P403', 'P501', 'R401', '232', '233', '234', 'C401', 'F401', 'PR512', 'R514', 'R516', 'RC301', 'M401', 'M405', 'P201', 'P403', 'P411', 'R513', '232', 'B101', 'B102', 'B404', 'C402', 'B101', 'B102', 'M702', 'P511', 'C411', 'C511', 'CR402', '233', 'P201', 'P511']


In [20]:
product_list=[]
i=1
while True:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_index=r.get('https://www.urcosme.com/tags/138/products?channel=0&effect=0&texture=0&modus=0&market_date=0&is_limit=false&is_discontinued=false&is_withdraw=false&is_cosme_store=false&price_min=&price_max=&sort=2&page=%s'%i,headers=headers)
    soup=BeautifulSoup(product_index.text,'html5lib')
    if soup.select('div.uc-product-item > div.product-infomation > div.product-name > a') == []:
        break
    for page in soup.select('div.uc-product-item > div.product-infomation > div.product-name > a'):
        product_list.append('https://www.urcosme.com'+page['href'])
        print(page)
    print(i)
    i+=1

<a class="uc-content-link single-dot" href="/products/49346">情挑誘光水唇膏</a>
<a class="uc-content-link single-dot" href="/products/75932">好氣色漸層三色CC輕唇膏</a>
<a class="uc-content-link single-dot" href="/products/45651">奶油潤彩亮唇膏</a>
<a class="uc-content-link single-dot" href="/products/12176">超炫耀的唇膏 ROUGE ALLURE</a>
<a class="uc-content-link single-dot" href="/products/60643">絕色濃唇膏</a>
<a class="uc-content-link single-dot" href="/products/34680">膠原豐潤唇膏</a>
<a class="uc-content-link single-dot" href="/products/77760">極綻色柔霧花蜜唇膏</a>
<a class="uc-content-link single-dot" href="/products/58054">純色訂製唇膏 COLOR RICH MOIST MATTE</a>
<a class="uc-content-link single-dot" href="/products/71015">純真唇頰彩</a>
<a class="uc-content-link single-dot" href="/products/15174">晶透唇膏</a>
1
<a class="uc-content-link single-dot" href="/products/38289">COCO水亮唇膏 ROUGE COCO SHINE</a>
<a class="uc-content-link single-dot" href="/products/6328">時尚專業唇膏 LIPSTICK</a>
<a class="uc-content-link single-dot" href="/products/49445">戀愛持

<a class="uc-content-link single-dot" href="/products/17304">潤采唇膏 Creamy Lip Color</a>
<a class="uc-content-link single-dot" href="/products/56333">LED超誘光精萃唇膏 Serum Intense Lipstick</a>
<a class="uc-content-link single-dot" href="/products/56679">微笑舒芙蕾粉漾唇彩</a>
<a class="uc-content-link single-dot" href="/products/76665">藍星唇膏</a>
<a class="uc-content-link single-dot" href="/products/61063">縱慾惹火唇膏</a>
<a class="uc-content-link single-dot" href="/products/30629">心機水光誘唇膏</a>
<a class="uc-content-link single-dot" href="/products/43219">上質光．豐潤持色保養唇膏</a>
<a class="uc-content-link single-dot" href="/products/44755">極綻色漾采唇膏 Color Sensational lipsticks</a>
<a class="uc-content-link single-dot" href="/products/9741">悅虹唇膏 Lip Color</a>
<a class="uc-content-link single-dot" href="/products/77848">無色限粉霧保濕唇膏</a>
11
<a class="uc-content-link single-dot" href="/products/16769">時尚經典唇膏 Lipstick</a>
<a class="uc-content-link single-dot" href="/products/42398">超炫耀的絲絨唇膏</a>
<a class="uc-content-link single-

<a class="uc-content-link single-dot" href="/products/50130">香吻誘惑唇膏 Le Rouge Givenchy</a>
<a class="uc-content-link single-dot" href="/products/79361">口紅</a>
<a class="uc-content-link single-dot" href="/products/35101">烈吻色唇膏 Rouge Artist Intense</a>
<a class="uc-content-link single-dot" href="/products/68149">紐約普普塗鴉唇膏 Pop Lip Colour </a>
<a class="uc-content-link single-dot" href="/products/22581">一代尤霧絲柔唇霜 So D&amp;eacute;licate Lipcream</a>
<a class="uc-content-link single-dot" href="/products/78178">城市CC持色唇膏</a>
<a class="uc-content-link single-dot" href="/products/18808">3D美型持久口紅(GLOW) Full memory rouge (glow)</a>
<a class="uc-content-link single-dot" href="/products/77459">熊大霧感胭脂唇膏SPF13</a>
<a class="uc-content-link single-dot" href="/products/74402">極綻色烈焰放肆絲絨霧光唇膏</a>
<a class="uc-content-link single-dot" href="/products/74468">小黑裙我的香氛唇膏</a>
21
<a class="uc-content-link single-dot" href="/products/21995">牛奶滋潤唇膏</a>
<a class="uc-content-link single-dot" href="/products/67591">特潤蜜唇膏<

<a class="uc-content-link single-dot" href="/products/79386">精萃幻影唇膏 Herbal Lipstick</a>
<a class="uc-content-link single-dot" href="/products/59930">菲菲潤色唇膏 Lip Tint Hydrator</a>
<a class="uc-content-link single-dot" href="/products/78369">水吻炫彩觸控唇膏</a>
<a class="uc-content-link single-dot" href="/products/5669">魅力口紅N RMK Irresistible Lips</a>
<a class="uc-content-link single-dot" href="/products/5414">煽動恆燦唇膏</a>
<a class="uc-content-link single-dot" href="/products/52778">法式之吻唇膏 French Kiss</a>
<a class="uc-content-link single-dot" href="/products/78639">WINE LIP STICK</a>
<a class="uc-content-link single-dot" href="/products/14640">水潤唇彩N LIP COLOR N</a>
<a class="uc-content-link single-dot" href="/products/77894">新禁忌之吻唇膏 ROUGUE INTERDIT</a>
<a class="uc-content-link single-dot" href="/products/38215">魅光蜜口紅 Glam Touch Lipstick Sheer</a>
31
<a class="uc-content-link single-dot" href="/products/78922">癮炫持久唇膏 LIPSTICK PURE</a>
<a class="uc-content-link single-dot" href="/products/8524">8小時

<a class="uc-content-link single-dot" href="/products/12801">benefit唇膏</a>
<a class="uc-content-link single-dot" href="/products/78849">ROUGE HOLIC EXCEPTIONAL</a>
<a class="uc-content-link single-dot" href="/products/71940">誘光唇膏 MYTHIC ROUGE</a>
<a class="uc-content-link single-dot" href="/products/35337">紅寶之吻水曜光唇膏(聖誕彩妝限量色) Rouge G Brillant</a>
<a class="uc-content-link single-dot" href="/products/75276">植物樂園玫瑰花開瑩潤唇膏</a>
<a class="uc-content-link single-dot" href="/products/13587">漾彩唇膏 Stick Gloss</a>
<a class="uc-content-link single-dot" href="/products/44311">色癮唇膏</a>
<a class="uc-content-link single-dot" href="/products/50970">驚綺絨彩筆</a>
<a class="uc-content-link single-dot" href="/products/30">潤炫唇膏8色 Lip stick</a>
<a class="uc-content-link single-dot" href="/products/73246">Q10水唇膏(保養型)</a>
41
<a class="uc-content-link single-dot" href="/products/67370">水潤唇筆</a>
<a class="uc-content-link single-dot" href="/products/68138">BURBERRY KISSES唇膏 BURBERRY KISSES</a>
<a class="uc-content-li

<a class="uc-content-link single-dot" href="/products/30075">咖啡香唇膏</a>
<a class="uc-content-link single-dot" href="/products/44667">時尚色繪尚質光采唇膏 Perfect Rouge Tender Sheer</a>
<a class="uc-content-link single-dot" href="/products/383">雅芳水灩光唇膏</a>
<a class="uc-content-link single-dot" href="/products/7554">無吻痕水潤唇膏</a>
<a class="uc-content-link single-dot" href="/products/26245">瓷光亮采唇膏</a>
<a class="uc-content-link single-dot" href="/products/53127">烈豔風華紅石榴防曬唇膏SPF10</a>
<a class="uc-content-link single-dot" href="/products/83597">迷戀輕吻唇膏 Crushed Lip Color</a>
<a class="uc-content-link single-dot" href="/products/78224">櫻花光感4D唇膏 Moisture Care Lip Stick EX</a>
<a class="uc-content-link single-dot" href="/products/404">持久真采唇膏</a>
<a class="uc-content-link single-dot" href="/products/59030">愛之天使變色唇膏</a>
51
<a class="uc-content-link single-dot" href="/products/29598">絕美金璨限量版風華唇膏 Opulent Shimmer Gloss</a>
<a class="uc-content-link single-dot" href="/products/62879">超炫耀亮口紅 City chic lipstick</a>
<

<a class="uc-content-link single-dot" href="/products/8288">保濕潤澤口紅 COVERMARK REALFINISH LIPSTICK</a>
<a class="uc-content-link single-dot" href="/products/2150">恆彩銀亮唇膏 Color Ever Silvershine</a>
<a class="uc-content-link single-dot" href="/products/80998">리얼 컬러 립스틱</a>
<a class="uc-content-link single-dot" href="/products/60520">水潤亮彩唇膏 MISSHA M Glossy Lip Rouge SPF13</a>
<a class="uc-content-link single-dot" href="/products/59753">絲柔亮彩唇膏</a>
<a class="uc-content-link single-dot" href="/products/62826">Lip Color Matte </a>
<a class="uc-content-link single-dot" href="/products/4716">活顏悅色 恒采唇膏 The Makeup Staying Power Moisturizing Lipstick</a>
<a class="uc-content-link single-dot" href="/products/4205">水潤唇膏 Water Reflection Lipstick</a>
<a class="uc-content-link single-dot" href="/products/62579">心花朵朵開魔力之吻愛戀唇膏(閃閃動人限定版)</a>
<a class="uc-content-link single-dot" href="/products/30070">緞光巨星唇膏 Satin Lip Colour</a>
61
<a class="uc-content-link single-dot" href="/products/66423">吻色唇膏</a>
<a cla

<a class="uc-content-link single-dot" href="/products/66564">Max貓小唇膏</a>
<a class="uc-content-link single-dot" href="/products/75525">晶巧澄色美顏棒 LUNASOL CLEAR COLOR STICK</a>
<a class="uc-content-link single-dot" href="/products/30982">AD奢華煥彩唇膏</a>
<a class="uc-content-link single-dot" href="/products/15112">KissKiss冰金極光唇膏</a>
<a class="uc-content-link single-dot" href="/products/67593">Babyface Candy Glow Tint Bar</a>
<a class="uc-content-link single-dot" href="/products/66570">棒棒糖防曬潤唇膏SPF13 Lolly Pop SPF13</a>
<a class="uc-content-link single-dot" href="/products/67594">It’s Top High Glossy Lipstick</a>
<a class="uc-content-link single-dot" href="/products/81931">水凝緋紅唇膏</a>
<a class="uc-content-link single-dot" href="/products/66572">華麗搖滾口紅 GLAM  ROCK  Vampire's  Kiss</a>
<a class="uc-content-link single-dot" href="/products/64013">雙面女郎唇膏指彩組 SEXY BACK DUET</a>
71
<a class="uc-content-link single-dot" href="/products/46862">純色晶灩謎幻光唇膏 Pure Color Vivid Shine Lipstick</a>
<a class="uc-conte

<a class="uc-content-link single-dot" href="/products/52319">Passion Duo Gloss Fusion Lipstick</a>
<a class="uc-content-link single-dot" href="/products/52320">Femme Rouge Velvet Creme Lipstick</a>
<a class="uc-content-link single-dot" href="/products/8289">水亮潤澤口紅 COVERMARK REALFINISH LIPSTICK N</a>
<a class="uc-content-link single-dot" href="/products/48994">保濕絕色唇膏 </a>
<a class="uc-content-link single-dot" href="/products/63331">聖誕寶貝潤色護唇膏 Santa Baby</a>
<a class="uc-content-link single-dot" href="/products/77412">幻粧釉光蜜唇膏</a>
<a class="uc-content-link single-dot" href="/products/76389">癮誘超模唇膏(巴黎天空限定版)</a>
<a class="uc-content-link single-dot" href="/products/34405">紅磨坊限量版唇膏 ROUGE ARTIST LIPSTICK SHADE MOULIN ROUGE</a>
<a class="uc-content-link single-dot" href="/products/75366">라비오뜨 페탈 어페어 립 컬러 에센스 슬림핏 PETAL AFFAIR LIP COLOR ESSENCE SLIM FIT</a>
<a class="uc-content-link single-dot" href="/products/21350">倩碧唇膏 Different Lipstick</a>
81
<a class="uc-content-link single-dot" href="/prod

<a class="uc-content-link single-dot" href="/products/80078">メスメリック リップスティック</a>
<a class="uc-content-link single-dot" href="/products/39630">晶蜜唇膏 Gel Lip Colour</a>
<a class="uc-content-link single-dot" href="/products/67281">持久鎖色盈潤唇膏</a>
<a class="uc-content-link single-dot" href="/products/19667">花妍輕舞曲-水潤光唇膏 TFS REAL MOISTURE LIPSTICK</a>
<a class="uc-content-link single-dot" href="/products/79828">마음의 소리 틴트 밤</a>
<a class="uc-content-link single-dot" href="/products/18900">純色晶亮旋彩持久唇膏 Pure Color Lipstick</a>
<a class="uc-content-link single-dot" href="/products/19668">花妍輕舞曲－迷霧光唇膏 TFS REAL MATTE LIPSTICK</a>
<a class="uc-content-link single-dot" href="/products/6102">JS蜜口紅</a>
<a class="uc-content-link single-dot" href="/products/79062">Full Kisses唇膏</a>
<a class="uc-content-link single-dot" href="/products/43736">WannaBe俏儷時尚～法式口感鮮唇膏</a>
91
<a class="uc-content-link single-dot" href="/products/49880">微笑貓咪光俏唇彩</a>
<a class="uc-content-link single-dot" href="/products/59608"> 繽紛森巴豐潤唇膏限

In [21]:
product_list

['https://www.urcosme.com/products/49346',
 'https://www.urcosme.com/products/75932',
 'https://www.urcosme.com/products/45651',
 'https://www.urcosme.com/products/12176',
 'https://www.urcosme.com/products/60643',
 'https://www.urcosme.com/products/34680',
 'https://www.urcosme.com/products/77760',
 'https://www.urcosme.com/products/58054',
 'https://www.urcosme.com/products/71015',
 'https://www.urcosme.com/products/15174',
 'https://www.urcosme.com/products/38289',
 'https://www.urcosme.com/products/6328',
 'https://www.urcosme.com/products/49445',
 'https://www.urcosme.com/products/12541',
 'https://www.urcosme.com/products/76619',
 'https://www.urcosme.com/products/43976',
 'https://www.urcosme.com/products/74401',
 'https://www.urcosme.com/products/77107',
 'https://www.urcosme.com/products/53250',
 'https://www.urcosme.com/products/66488',
 'https://www.urcosme.com/products/66752',
 'https://www.urcosme.com/products/64228',
 'https://www.urcosme.com/products/42501',
 'https://ww

In [29]:
a='/'
a=a.replace('/','b')
a

'b'

In [ ]:
#唇膏

import requests as r
import shutil
import splinter
from bs4 import BeautifulSoup
import os
import json
import csv





product_list=[]
i=1
while True:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_index=r.get('https://www.urcosme.com/tags/138/products?channel=0&effect=0&texture=0&modus=0&market_date=0&is_limit=false&is_discontinued=false&is_withdraw=false&is_cosme_store=false&price_min=&price_max=&sort=2&page=%s'%i,headers=headers)
    soup=BeautifulSoup(product_index.text,'html5lib')
    if soup.select('div.uc-product-item > div.product-infomation > div.product-name > a') == []:
        break
    for page in soup.select('div.uc-product-item > div.product-infomation > div.product-name > a'):
        product_list.append('https://www.urcosme.com'+page['href'])
#        print(page)
#    print(i)
    i+=1

    
for url in product_list:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_page=r.get(url,headers=headers)
    soup=BeautifulSoup(product_page.text,'html5lib')
    comment_list=[]


    product={}
    product['product_name']=soup.select('div.headline-title.product-name')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    
    product['brand']=soup.select('div.brand-name > a.uc-main-link')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    for i in range(0,len(soup.select('div.other-label'))):
        if soup.select('div.other-label')[i].text=='容量':
            product['volumn']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='價格':
            product['price']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='上市日期':
            product['listing_date']=soup.select('div.other-text')[i].text
    if soup.select('div.product-info-image > img.img-auto-center')[0]['src']=='https://dg9ugnb21lig7.cloudfront.net/uploads/images/product_default_250x250.png':
        product['pro_picture']='null'
    else:
        product['pro_picture']=soup.select('div.product-info-image > img.img-auto-center')[0]['src']
    product['label']=[]
    for i in soup.select('div.detail-text'):
        product['label'].append(i.text)
    try:
        product['pro_info']=soup.find('div','product-desc-content').text
    except:
        product['pro_info']='null'
    try:
        product['score']=soup.find('div','score-txt').text.split(' ')[1]
    except:
        product['score']='null'
    try:    
        if soup.select('div.uc-container-sub-title')[0].text=='色號':
            color_code = [i.text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','') for i in soup.select('div.color-tag-container > div.color-number-items > div.color-number-item > div.color-number-item-text > div.two-line-dot')]
        else:
            color_code=['one_color']
    except:
        color_code=['one_color']
    #if os.path.exists('D:\\產品資料\\產品\\唇膏\\'+product['brand']+'_'+product['product_name']+'_'++'.json'):
    #    continue    
    
    i=1
    comment_list=[]
    while True:
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comment_list_url=r.get(url+'/reviews?page=%s'%i,headers=headers)
        comment_soup=BeautifulSoup(comment_list_url.text,'html5lib')
        if comment_soup.select('div.uc-reviews > a') ==[]:
            break
        for a in comment_soup.select('div.uc-reviews > a'):
            comment_list.append('https://www.urcosme.com'+a['href'])
        i+=1
    
    print('comment_list complete')

    product['comments']=[]
    for count in range(0,len(comment_list)):
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comments=r.get(comment_list[count],headers=headers)
        soup=BeautifulSoup(comments.text,'html5lib')
        artist_INFO=soup.select('div.author-name > a')
        artist_ID=artist_INFO[0]['href'].split('/')[2]
        artist_name=artist_INFO[0].text
        review_content=soup.select('div.review-content')[0].text
        affect=soup.find('div','review-effects').text.split('：')[1].split('、')
        user_score=soup.find('div','review-score').text
        comments_published_time=soup.find('div','review-date-and-pageview').text.split(' ')[0]
        product['comments'].append({'user_ID':artist_ID,
                                     'content':review_content,
                                     'affect':affect,
                                     'user_score':user_score,
                                     'comments_published_time':comments_published_time})
    print('評論已完成')
    for col in color_code:
        product['product_color']=col
        with open('D:\\產品資料\\產品\\唇膏\\'+product['brand']+'_'+product['product_name']+'_'+col+'.json','w') as f :
            json.dump(product,f)

In [ ]:
#眉筆

import requests as r
import shutil
import splinter
from bs4 import BeautifulSoup
import os
import json
import csv





product_list=[]
i=1
while True:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_index=r.get('https://www.urcosme.com/tags/115/products?channel=0&effect=0&texture=0&modus=0&market_date=0&is_limit=false&is_discontinued=false&is_withdraw=false&is_cosme_store=false&price_min=&price_max=&sort=2&page=%s'%i,headers=headers)
    soup=BeautifulSoup(product_index.text,'html5lib')
    if soup.select('div.uc-product-item > div.product-infomation > div.product-name > a') == []:
        break
    for page in soup.select('div.uc-product-item > div.product-infomation > div.product-name > a'):
        product_list.append('https://www.urcosme.com'+page['href'])
#        print(page)
#    print(i)
    i+=1

    
for url in product_list:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_page=r.get(url,headers=headers)
    soup=BeautifulSoup(product_page.text,'html5lib')
    comment_list=[]


    product={}
    product['product_name']=soup.select('div.headline-title.product-name')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    
    product['brand']=soup.select('div.brand-name > a.uc-main-link')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    for i in range(0,len(soup.select('div.other-label'))):
        if soup.select('div.other-label')[i].text=='容量':
            product['volumn']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='價格':
            product['price']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='上市日期':
            product['listing_date']=soup.select('div.other-text')[i].text
    if soup.select('div.product-info-image > img.img-auto-center')[0]['src']=='https://dg9ugnb21lig7.cloudfront.net/uploads/images/product_default_250x250.png':
        product['pro_picture']='null'
    else:
        product['pro_picture']=soup.select('div.product-info-image > img.img-auto-center')[0]['src']
    product['label']=[]
    for i in soup.select('div.detail-text'):
        product['label'].append(i.text)
    try:
        product['pro_info']=soup.find('div','product-desc-content').text
    except:
        product['pro_info']='null'
    try:
        product['score']=soup.find('div','score-txt').text.split(' ')[1]
    except:
        product['score']='null'
    try:    
        if soup.select('div.uc-container-sub-title')[0].text=='色號':
            color_code = [i.text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','') for i in soup.select('div.color-tag-container > div.color-number-items > div.color-number-item > div.color-number-item-text > div.two-line-dot')]
        else:
            color_code=['one_color']
    except:
        color_code=['one_color']
    #if os.path.exists('D:\\產品資料\\產品\\唇膏\\'+product['brand']+'_'+product['product_name']+'_'++'.json'):
    #    continue    
    
    i=1
    comment_list=[]
    while True:
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comment_list_url=r.get(url+'/reviews?page=%s'%i,headers=headers)
        comment_soup=BeautifulSoup(comment_list_url.text,'html5lib')
        if comment_soup.select('div.uc-reviews > a') ==[]:
            break
        for a in comment_soup.select('div.uc-reviews > a'):
            comment_list.append('https://www.urcosme.com'+a['href'])
        i+=1
    
    print('comment_list complete')

    product['comments']=[]
    for count in range(0,len(comment_list)):
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comments=r.get(comment_list[count],headers=headers)
        soup=BeautifulSoup(comments.text,'html5lib')
        artist_INFO=soup.select('div.author-name > a')
        artist_ID=artist_INFO[0]['href'].split('/')[2]
        artist_name=artist_INFO[0].text
        review_content=soup.select('div.review-content')[0].text
        affect=soup.find('div','review-effects').text.split('：')[1].split('、')
        user_score=soup.find('div','review-score').text
        comments_published_time=soup.find('div','review-date-and-pageview').text.split(' ')[0]
        product['comments'].append({'user_ID':artist_ID,
                                     'content':review_content,
                                     'affect':affect,
                                     'user_score':user_score,
                                     'comments_published_time':comments_published_time})
    print('評論已完成')
    for col in color_code:
        product['product_color']=col
        with open('D:\\產品資料\\產品\\眉筆\\'+product['brand']+'_'+product['product_name']+'_'+col+'.json','w') as f :
            json.dump(product,f)

comment_list complete


In [31]:
#粉底液

import requests as r
import shutil
import splinter
from bs4 import BeautifulSoup
import os
import json
import csv





product_list=[]
i=1
while True:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_index=r.get('https://www.urcosme.com/tags/105/products?channel=0&effect=0&texture=0&modus=0&market_date=0&is_limit=false&is_discontinued=false&is_withdraw=false&is_cosme_store=false&price_min=&price_max=&sort=2&page=%s'%i,headers=headers)
    soup=BeautifulSoup(product_index.text,'html5lib')
    if soup.select('div.uc-product-item > div.product-infomation > div.product-name > a') == []:
        break
    for page in soup.select('div.uc-product-item > div.product-infomation > div.product-name > a'):
        product_list.append('https://www.urcosme.com'+page['href'])
#        print(page)
#   print(i)
    i+=1

    
for url in product_list:
    headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    product_page=r.get(url,headers=headers)
    soup=BeautifulSoup(product_page.text,'html5lib')
    comment_list=[]


    product={}
    product['product_name']=soup.select('div.headline-title.product-name')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    
    product['brand']=soup.select('div.brand-name > a.uc-main-link')[0].text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','')
    for i in range(0,len(soup.select('div.other-label'))):
        if soup.select('div.other-label')[i].text=='容量':
            product['volumn']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='價格':
            product['price']=soup.select('div.other-text')[i].text
        elif soup.select('div.other-label')[i].text=='上市日期':
            product['listing_date']=soup.select('div.other-text')[i].text
    if soup.select('div.product-info-image > img.img-auto-center')[0]['src']=='https://dg9ugnb21lig7.cloudfront.net/uploads/images/product_default_250x250.png':
        product['pro_picture']='null'
    else:
        product['pro_picture']=soup.select('div.product-info-image > img.img-auto-center')[0]['src']
    product['label']=[]
    for i in soup.select('div.detail-text'):
        product['label'].append(i.text)
    try:
        product['pro_info']=soup.find('div','product-desc-content').text
    except:
        product['pro_info']='null'
    try:
        product['score']=soup.find('div','score-txt').text.split(' ')[1]
    except:
        product['score']='null'
    try:    
        if soup.select('div.uc-container-sub-title')[0].text=='色號':
            color_code = [i.text.replace('?','').replace('\u3000','').replace('♥','').replace('@','').replace('"','').replace('.','').replace(':','').replace('!','').replace('/','').replace('*','').replace('<','').replace('>','').replace('(','').replace(')','').replace('\t','').replace('/','').replace('\n','') for i in soup.select('div.color-tag-container > div.color-number-items > div.color-number-item > div.color-number-item-text > div.two-line-dot')]
        else:
            color_code=['one_color']
    except:
        color_code=['one_color']
    #if os.path.exists('D:\\產品資料\\產品\\唇膏\\'+product['brand']+'_'+product['product_name']+'_'++'.json'):
    #    continue    
    
    i=1
    comment_list=[]
    while True:
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comment_list_url=r.get(url+'/reviews?page=%s'%i,headers=headers)
        comment_soup=BeautifulSoup(comment_list_url.text,'html5lib')
        if comment_soup.select('div.uc-reviews > a') ==[]:
            break
        for a in comment_soup.select('div.uc-reviews > a'):
            comment_list.append('https://www.urcosme.com'+a['href'])
        i+=1
    
    print('comment_list complete')

    product['comments']=[]
    for count in range(0,len(comment_list)):
        headers={'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        comments=r.get(comment_list[count],headers=headers)
        soup=BeautifulSoup(comments.text,'html5lib')
        artist_INFO=soup.select('div.author-name > a')
        artist_ID=artist_INFO[0]['href'].split('/')[2]
        artist_name=artist_INFO[0].text
        review_content=soup.select('div.review-content')[0].text
        affect=soup.find('div','review-effects').text.split('：')[1].split('、')
        user_score=soup.find('div','review-score').text
        comments_published_time=soup.find('div','review-date-and-pageview').text.split(' ')[0]
        product['comments'].append({'user_ID':artist_ID,
                                     'content':review_content,
                                     'affect':affect,
                                     'user_score':user_score,
                                     'comments_published_time':comments_published_time})
    print('評論已完成')
    for col in color_code:
        product['product_color']=col
        with open('D:\\產品資料\\產品\\粉底液\\'+product['brand']+'_'+product['product_name']+'_'+col+'.json','w') as f :
            json.dump(product,f)

comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complet

評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list c

comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complete
評論已完成
comment_list complet